# 롸켓펀치 크롤링

In [4]:
import scrapy
from scrapy.http import TextResponse

In [46]:
# __init__
# start url
url = "https://www.rocketpunch.com/api/jobs/template?&page=1&q=&specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0"

# get_pages 함수
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
response

<200 https://www.rocketpunch.com/api/jobs/template?&page=1&q=&specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0>

  company_name = scrapy.Field()
    business = scrapy.Field()
    position = scrapy.Field()
    link = scrapy.Field()
    salary_condition = scrapy.Field()
    deadline = scrapy.Field()
    keyword = scrapy.Field()
    location = scrapy.Field()

In [48]:
# company_name
response.text

'{"meta":{"code":200},"data":{"template":"<div id=\\"company-list\\" class=\\"ui job items segment\\">\\n\\n  <div class=\\"header\\">\\n    <div class=\\"ui right floated header sort-types\\"><a data-query-add=\\"sort=recent\\" href=\\"/jobs?q=&amp;specialty=Python&amp;tag=%EB%8D%B0%EC%9D%B4%ED%84%B0&amp;sort=recent\\"class=\\"checked\\">\\n        최신순\\n      </a>\\n<div class=\\"separator\\"></div><a data-query-add=\\"sort=ranking\\" href=\\"/jobs?q=&amp;specialty=Python&amp;tag=%EB%8D%B0%EC%9D%B4%ED%84%B0&amp;sort=ranking\\">\\n        인기순\\n      </a>\\n<div class=\\"separator\\"></div><a data-query-add=\\"sort=salary\\" href=\\"/jobs?q=&amp;specialty=Python&amp;tag=%EB%8D%B0%EC%9D%B4%ED%84%B0&amp;sort=salary\\">\\n        연봉순\\n      </a>\\n</div>\\n\\n    <h2 class=\\"ui left header\\" style=\\"font-size:16px\\">\\n      <span class=\\"desc\\">검색 결과</span>\\n      <span class=\\"count\\">58</span>\\n        <div class=\\"separator\\"></div>\\n        <span class=\\"filter-summar

In [31]:
# business
response.xpath('//*')

[<Selector xpath='//*' data='<html><body><div id="company-list" cl...'>,
 <Selector xpath='//*' data='<body><div id="company-list" class="u...'>,
 <Selector xpath='//*' data='<div id="company-list" class="ui job ...'>,
 <Selector xpath='//*' data='<div class="header">\n    <div class="...'>,
 <Selector xpath='//*' data='<div class="ui right floated header s...'>,
 <Selector xpath='//*' data='<a data-query-add="sort=recent" href=...'>,
 <Selector xpath='//*' data='<div class="separator"></div>'>,
 <Selector xpath='//*' data='<a data-query-add="sort=ranking" href...'>,
 <Selector xpath='//*' data='<div class="separator"></div>'>,
 <Selector xpath='//*' data='<a data-query-add="sort=salary" href=...'>,
 <Selector xpath='//*' data='<h2 class="ui left header" style="fon...'>,
 <Selector xpath='//*' data='<span class="desc">검색 결과</span>'>,
 <Selector xpath='//*' data='<span class="count">58</span>'>,
 <Selector xpath='//*' data='<div class="separator"></div>'>,
 <Selector xpath='//*' data='<

In [336]:
len(dom.select('#company-list > div.company.item > div.content > div.company-jobs-detail > div:nth-child(1) > div:nth-child(1) > a'))

40

In [337]:
len(dom.select('#company-list > div.company.item > div.content > div.company-jobs-detail > div.job-detail > div:nth-child(1) > a'))

56

In [290]:
dom.select('#company-list > div.company.item > div.content')[2].select('div.company-name > a > h4 > strong')[0].text

'아이디브'

In [294]:
dom.select('#company-list > div.company.item > div.content')[1].select('div.description')[0].text.strip('\xa0')

'페칭 - 테크 기반의 문화와 자유로운 분위기를 지향하는 회사'

In [497]:
def rocketpunch_crawler2(pages):
    datas = []
    for page in range(1,pages+1):
        print(page)
        url = "https://www.rocketpunch.com/api/jobs/template?page={}&q=&specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0%3E".format(page)
        response = requests.get(url)
        data = response.json()['data']['template']
        dom = BeautifulSoup(data, "html.parser")
        root = dom.select('#company-list > div.company.item > div.content')
        for i in range(20):
            datas.append({
                "company" : root[i].select('div.company-name > a > h4 > strong')[0].text,
                "description" : root[i].select('div.description')[0].text.strip('\xa0'),
                "service" : root[i].select('div.nowrap.meta')[0].text.strip(),
                "position" : root[i].select('div.company-jobs-detail > div:nth-child(1) > div:nth-child(1) > a')[0].text,
                "link" : "https://www.rocketpunch.com"+ root[i].select('div.company-jobs-detail > div:nth-child(1) > div:nth-child(1) > a')[0].get('href'),
                "salary" : root[i].select('div.company-jobs-detail > div:nth-child(1) > div:nth-child(1) > span')[0].text,
                "deadline" : root[i].select('div.company-jobs-detail > div.job-detail > div.job-dates > span:nth-child(1)')[0].text.strip(' \n'),
            })
            
    return pd.DataFrame(datas)

In [270]:
def rocketpunch_crawler(pages):
    datas = []
    for page in range(1,pages+1):
        print(page)
        url = "https://www.rocketpunch.com/api/jobs/template?page={}&q=&specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0%3E".format(page)
        response = requests.get(url)
        data = response.json()['data']['template']
        dom = BeautifulSoup(data, "html.parser")
        for i in range(20):
            datas.append({
                "company" : dom.select('#company-list > div.company.item > div.content > div.company-name > a > h4 > strong')[i].text,
                "description" : dom.select('#company-list > div.company.item > div.content > div.description')[i].text.strip('\xa0'),
                "service" : dom.select('#company-list > div.company.item > div.content > div.nowrap.meta')[i].text.strip(),
                "position" : dom.select('#company-list > div.company.item > div.content > div.company-jobs-detail > div:nth-child(1) > div:nth-child(1) > a')[i].text,
                "link" : "https://www.rocketpunch.com"+ dom.select('#company-list > div.company.item > div.content > div.company-jobs-detail > div:nth-child(1) > div:nth-child(1) > a')[0].get('href'),
                "salary" : dom.select('#company-list > div.company.item > div.content > div.company-jobs-detail > div:nth-child(1) > div:nth-child(1) > span')[i].text,
                "deadline" : dom.select('#company-list > div.company.item > div.content > div.company-jobs-detail > div.job-detail > div.job-dates > span:nth-child(1)')[i].text.strip(' \n'),
            })
            
    return pd.DataFrame(datas)

In [32]:
url = "https://www.rocketpunch.com/jobs?specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0"

In [33]:
response = requests.get(url)
response

<Response [200]>

In [34]:
response.text

'<!DOCTYPE html>\n<!--[if IE 9]>\n<html xmlns:fb="http://ogp.me/ns/fb#" class="ie ie9">\n<![endif]-->\n<!--[if (gt IE 9)|!(IE)]><!-->\n<html xmlns:fb="http://ogp.me/ns/fb#">\n<!--<![endif]-->\n<head>\n  <meta http-equiv="content-type" content="text/html; charset=utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=Edge">\n  <title>Python, 데이터 채용 정보\n | 로켓펀치 - 비즈니스 네트워크</title>\n<meta name="viewport" content="width=device-width,minimum-scale=1.0,initial-scale=1.0,maximum-scale=1.0,user-scalable=no">  <meta name="description" content="Python, 데이터 관련 채용 정보를 찾아보세요. \'로켓펀치\'는 비즈니스와 커리어를 성장시킬 수 있는 국내 최대 비즈니스 네트워크입니다.">\n<link rel="canonical" href="https://www.rocketpunch.com/jobs/tags/python" />\n<link rel="icon" href="/favicon.ico" type="image/x-icon">\n  <link rel="shortcut icon" href="/favicon.ico" type="image/x-icon">\n  <link rel="icon" type="image/png" href="/favicon-16x16.png" sizes="16x16">\n  <link rel="icon" type="image/png" href="/favicon-32x32.png" sizes="32x32">\n  <link re

In [541]:
# json으로 파싱후, data안 template 안에 들어있는 애들을 BrautifulSoup으로 html로 파싱
data = response.json()['data']['template']
dom = BeautifulSoup(data, "html.parser")

In [542]:
root = dom.select('#company-list > div.company.item > div.content')

In [543]:
# 회사명
dom.select('#company-list > div.company.item > div.content > div.company-name > a > h4 > strong')[0].text

'매스프레소'

In [544]:
root[0].select('div.company-name > a > h4 > strong')[0].text

'매스프레소'

In [521]:
# 회사 설명
dom.select('#company-list > div.company.item > div.content > div.description')[0].text.strip('\xa0')

'We Rebuild Education'

In [545]:
root[0].select('div.description')[0].text.strip('\xa0')

'We Rebuild Education'

In [523]:
# 분야
dom.select('#company-list > div.company.item > div.content > div.nowrap.meta')[0].text.strip()

'모바일서비스 ∙ 소프트웨어 ∙ 데이터 ∙ 교육 ∙ 모바일앱 ∙ AI ∙ 앱개발 ∙ 머신러닝 ∙ IT/인공지능 ∙ 딥러닝 ∙ 에듀테크 ∙ 수학 ∙ OCR'

In [546]:
root[0].select('div.nowrap.meta')[0].text.strip()

'모바일서비스 ∙ 소프트웨어 ∙ 데이터 ∙ 교육 ∙ 모바일앱 ∙ AI ∙ 앱개발 ∙ 머신러닝 ∙ IT/인공지능 ∙ 딥러닝 ∙ 에듀테크 ∙ 수학 ∙ OCR'

In [566]:
len(root)

20

In [525]:
# 중복 있음

In [556]:
positions = dom.select('#company-list > div.company.item > div.content > div.company-jobs-detail')

In [558]:
len_of_positions = len(positions[0].select('div:nth-child(1) > a.nowrap'))

In [559]:
len_of_positions

1

In [587]:
# position
positions[3].select('div:nth-child(1) > a.nowrap')[0].text

'해외직구 애프터바이 [Python,React 개발자] 모십니다!'

In [561]:
# 링크
"https://www.rocketpunch.com" + positions[0].select('div:nth-child(1) > a.nowrap')[0].get('href')

'https://www.rocketpunch.com/jobs/70589/Math-Engine-Developer'

In [562]:
# 연봉 / 스톡옵션 / 경력 여부
positions[0].select('div.job-detail > div:nth-child(1) > span')[0].text

'3,500 - 6,000만원 / 인턴, 신입, 경력'

In [563]:
# 채용 기간
positions[0].select('div.job-detail > div.job-dates > span:nth-child(1)')[0].text.strip(' \n')

'03/31 마감'

In [581]:
list(range(len_of_positions))

[0]

In [594]:
positions[18].select('div:nth-child(1) > a.nowrap')[0].text

'Backend Engineer 주니어'

In [613]:
root[4].select('div.company-name > a > h4 > strong')[0].text

'디에스랩글로벌'

In [614]:
positions[4].select('div:nth-child(1) > a.nowrap')[0].text

'데이터 분석가'

In [631]:
len_of_positions = len(positions[1].select('div:nth-child(1) > a.nowrap'))

In [632]:
list(range(len_of_positions))

[0, 1, 2]

In [636]:
root[1].select('div.company-name > a > h4 > strong')[0].text,

('트릿지',)

In [8]:
url = "https://www.rocketpunch.com/api/jobs/template?&page=1&q=&specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0"
response = requests.get(url)
data = response.json()['data']['template']
dom = BeautifulSoup(data, "html.parser")
total_pages = dom.select("div.ui.pagination.menu > div.tablet.computer.large.screen.widescreen.only > a")[-1].text

In [9]:
total_pages

'3'

In [827]:
# 크롤링 완성

In [2]:
import time
from datetime import date, timedelta

In [3]:
# 전체 페이지의 개수를 알아내기 위한 과정
url = "https://www.rocketpunch.com/api/jobs/template?&page=1&q=&specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0"
response = requests.get(url)
data = response.json()['data']['template']
dom = BeautifulSoup(data, "html.parser")
total_pages = dom.select("div.ui.pagination.menu > div.tablet.computer.large.screen.widescreen.only > a")[-1].text

datas = []
for page in range(1,int(total_pages)+1):
    url = "https://www.rocketpunch.com/api/jobs/template?page={}&q=&specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0%".format(page)
    response = requests.get(url)
    data = response.json()['data']['template']
    dom = BeautifulSoup(data, "html.parser")
    root = dom.select('#company-list > div.company.item > div.content')
    positions = dom.select('#company-list > div.company.item > div.content > div.company-jobs-detail')
    a = 0
    for i in range(len(root)):
        len_of_positions = len(positions[i].select('div:nth-child(1) > a.nowrap'))   
        a +=1
        print(a)
        for j in range(len_of_positions):
            link = "https://www.rocketpunch.com" + positions[i].select('div:nth-child(1) > a.nowrap')[j].get('href') 
            response = requests.get(link)
            dom_for_skills = BeautifulSoup(response.text,"html.parser")      
            datas.append({
                "date" : date.today(),
                "company" : root[i].select('div.company-name > a > h4 > strong')[0].text,
                "position" : positions[i].select('div:nth-child(1) > a.nowrap')[j].text,
                "salary" : positions[i].select('div.job-detail > div:nth-child(1) > span')[j].text,
                "deadline" : positions[i].select('div.job-detail > div.job-dates > span:nth-child(1)')[j].text.strip(' \n'),
                "description" : root[i].select('div.description')[0].text.strip('\xa0'),
                "service" : root[i].select('div.nowrap.meta')[0].text.strip(),  
                "link" : link,
                "skills" : str([a.text for a in dom_for_skills.select('div .job-specialties > a ')]).strip("[]")
            })
result = pd.DataFrame(datas)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
1
2
3
4
5
6
7
8
9


,date,company,position,salary,deadline,description,service,link,skills
0,2020-03-11,오르비스AI,(경력) Data Scientist,"5,000 - 6,000만원 / 최소 0.1% / 경력",04/30 마감,영어권 컨텐츠 및 광고 크리에이터를 위한 AI 성우를 개발합니다.,IT ∙ 교육 ∙ 인공지능 ∙ 머신러닝 ∙ 딥러닝 ∙ MachineLearning ...,https://www.rocketpunch.com/jobs/47245/%EA%B2%...,[]
1,2020-03-11,무른모,C# 개발자(경력/신입),"신입, 경력",수시채용,"4차산업 혁명을 이끌 ICT, IOT 기반 자동차 부품제조업 MES 솔루션 개발업체",ICT ∙ 클라우드서비스 ∙ IoT플랫폼 ∙ MES ∙ 스마트공장운영시스템 ∙ LO...,https://www.rocketpunch.com/jobs/70735/C-%EA%B...,"[MySQL, plc, PHP, aws, GitLab, Python]"
2,2020-03-11,네이콘,PHP/MY-SQL 웹프로그래머 개발자,"인턴, 신입, 경력",04/10 마감,글로벌 퍼포먼스 및 인플루언서 마케팅 플랫폼 회사,모바일광고 ∙ 퍼포먼스마케팅 ∙ 인플루언서마케팅 ∙ 동영상광고플랫폼 ∙ 애드네트웍크,https://www.rocketpunch.com/jobs/70733/PHPMY-S...,"[Android, Java, JavaScript, PHP, Python, Amazo..."
3,2020-03-11,에이아이메이커,"[java, python] 영상 Tool 개발","신입, 경력",수시채용,데이터 라벨링 기업,개발 ∙ IT/영상 ∙ 정보통신업/ ∙ 정보통신업/소프트웨어개발및공급업,https://www.rocketpunch.com/jobs/70163/java-py...,"[Python, Java, Amazon AWS, JavaScript, GitHub]"
4,2020-03-11,트릿지,Front-End Software Engineer at Tridge,"4,000 - 10,000만원 / 인턴, 신입, 경력",수시채용,Disrupting the $1.5 Trillion global trading in...,모바일서비스 ∙ 웹서비스 ∙ 마케팅 ∙ 스타트업 ∙ e-commerce ∙ 공유경제...,https://www.rocketpunch.com/jobs/70411/Front-E...,"[AngularJS, jQuery, PostgreSQL, nginx, Python,..."
...,...,...,...,...,...,...,...,...,...
314,2020-03-11,키인사이드,[상시채용] 키인사이드와 함께 할 블록체인 Web Service 개발자 모집,"3,000 - 10,000만원 / 경력",수시채용,Leading the way inside to enterprise blockchain,블록체인 ∙ blockchain ∙ 블록체인플랫폼,https://www.rocketpunch.com/jobs/66224/%EC%83%...,"[Android, iOS, Node.js, Java, golang, C#, Pyth..."
315,2020-03-11,플랜아이,너드팩토리 AI 추천시스템 엔지니어 영입,경력,수시채용,,"IT ∙ 솔루션,인공지능,데이터마이닝 ∙ B2G",https://www.rocketpunch.com/jobs/65167/%EB%84%...,"[Python, Flask, TensorFlow, PyTorch, 머신러닝, 인공지..."
316,2020-03-11,힐링사운드,연구원,"1.0% - 5.0% / 인턴, 신입, 경력",수시채용,의료용 의사소통 증진 시스템,의료 ∙ 치과,https://www.rocketpunch.com/jobs/65932/%EC%97%...,[더 보기]
317,2020-03-11,링크샵스,[링크샵스] 플랫폼 개발,"신입, 경력",수시채용,Global Fashion B2B Online Marketplace,모바일서비스 ∙ 웹서비스 ∙ 소프트웨어 ∙ 이커머스 ∙ B2B ∙ 이커머스/패션뷰티...,https://www.rocketpunch.com/jobs/65771/%EB%A7%...,"[플랫폼, 웹 어플리케이션 설계/개발 (Java/JSP), 데이터분석, 알고리즘, ..."


In [86]:
[a.text for a in dom_for_skills.select('#wrap > div.eight.wide.job-content.column > section:nth-child(3) > div > a')] 

['C',
 'C++',
 'Mac OS X',
 'Qt',
 '3D CAD',
 'Python',
 'Xcode',
 'Visual Studio',
 'SVN',
 'Git',
 'TeamCity']

In [87]:
[a.text for a in dom_for_skills.select('div .job-specialties')]

['\nC\nC++\nMac OS X\nQt\n3D CAD\nPython\nXcode\nVisual Studio\nSVN\nGit\nTeamCity\n',
 '\n          Front-end ∙ JavaScript ∙ jQuery ∙ MySQL ∙ Java ∙ Node.js ∙ React ∙ CSS3 ∙ Android ∙ iOS\n        ',
 '\n          jQuery ∙ JavaScript ∙ CSS3 ∙ AngularJS ∙ React ∙ Front-end ∙ HTML5 ∙ CSS ∙ HTML ∙ Node.js ∙ MySQL ∙ Java\n        ',
 '\n          unity ∙ Unity 3D ∙ Unreal ∙ 유니티 ∙ shader ∙ 3D ∙ Unreal Engine 4 ∙ ar ∙ vr\n        ',
 '\n          3D디자인 ∙ 3d MAYA ∙ 패션디자인 ∙ 패션 ∙ 프로덕트디자인\n        ']

In [286]:
response = requests.get("https://www.rocketpunch.com/jobs/57705/Back-end-Developer-%EB%B0%B1%EC%97%94%EB%93%9C-%EC%84%9C%EB%B2%84-%EA%B0%9C%EB%B0%9C%EC%9E%90")
dom = BeautifulSoup(response.text,"html.parser")

In [920]:
response = requests.get("https://www.rocketpunch.com/jobs/70804/%EC%B2%AD%EB%85%84SW%EC%95%84%EC%B9%B4%EB%8D%B0%EB%AF%B8SSAFY-%EC%B7%A8%EC%97%85%EC%83%81%EB%8B%B4%EC%82%AC-%EA%B3%84%EC%95%BD%EC%A7%81-%EB%8C%80%EC%A0%84-%EA%B4%91%EC%A3%BC")
dom_for_skills = BeautifulSoup(response.text,"html.parser")      

In [992]:
", ".join([a.text for a in dom_for_skills.select('div .job-specialties > a ')])

'취업상담사, 취업상담'

In [995]:
# <br\>로 나누어져 있기 때문에, 처음부터 끝까지 1칸씩 간격을 두고 찾아내면 지역이 몇개건 상관없이 가져올수 있음
", ".join(dom_for_skills.select('#wrap > div.four.wide.job-infoset.column > div > div:nth-child(3) > div > div:nth-child(2) > div.content')[0].contents[::2]).strip()

'광주광역시 광산구, 대전광역시 유성구'

In [996]:
"\n       광주광역시                 \n".strip()

'광주광역시'

In [40]:
from scrapy.http import TextResponse
url = "https://www.rocketpunch.com/api/jobs/template?page=1&q=&specialty=Python&tag=데이터"

In [82]:
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://www.rocketpunch.com/api/jobs/template?page=1&q=&specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0%3E>

In [83]:
jsonresponse = json.loads(response.body)

In [72]:
soup = BeautifulSoup(response.body, "html.parser") 

In [75]:
soup

TypeError: 'NoneType' object is not callable

In [48]:
url = "https://www.rocketpunch.com/api/jobs/template?page=1&q=&specialty=Python&tag=%EB%8D%B0%EC%9D%B4%ED%84%B0%3E"
response = requests.get(url)
data = response.json()['data']['template']
dom = BeautifulSoup(data, "html.parser")


In [49]:
dom

<div class="ui job items segment" id="company-list">
<div class="header">
<div class="ui right floated header sort-types"><a class="checked" data-query-add="sort=recent" href="/jobs?q=&amp;specialty=Python&amp;tag=%EB%8D%B0%EC%9D%B4%ED%84%B0%3E&amp;sort=recent">
        최신순
      </a>
<div class="separator"></div><a data-query-add="sort=ranking" href="/jobs?q=&amp;specialty=Python&amp;tag=%EB%8D%B0%EC%9D%B4%ED%84%B0%3E&amp;sort=ranking">
        인기순
      </a>
<div class="separator"></div><a data-query-add="sort=salary" href="/jobs?q=&amp;specialty=Python&amp;tag=%EB%8D%B0%EC%9D%B4%ED%84%B0%3E&amp;sort=salary">
        연봉순
      </a>
</div>
<h2 class="ui left header" style="font-size:16px">
<span class="desc">검색 결과</span>
<span class="count">304</span>
<div class="separator"></div>
<span class="filter-summary">데이터&gt;, Python</span>
</h2>
</div>
<div class="ui inverted divider"></div><div class="company item active" data-company_id="113474">
<div class="logo image">
<a href="/companies

In [720]:
type(dom)

bs4.BeautifulSoup

In [721]:
type(response)

scrapy.http.response.text.TextResponse

In [708]:
response.xpath('#company-list > div.company.item.active > div.content.active > div.company-name > a:nth-child(1) > h4')

NotSupported: Response content isn't text

In [691]:
response.xpath('//*')

[<Selector xpath='//*' data='<html><body><p>{"meta":{"code":200},"...'>,
 <Selector xpath='//*' data='<body><p>{"meta":{"code":200},"data":...'>,
 <Selector xpath='//*' data='<p>{"meta":{"code":200},"data":{"temp...'>,
 <Selector xpath='//*' data='<div id=\'\\"company-list\\"\' class=\'\\"u...'>,
 <Selector xpath='//*' data='<div class=\'\\"header\\"\'>\\n    <div cl...'>,
 <Selector xpath='//*' data='<div class=\'\\"ui\' right floated heade...'>,
 <Selector xpath='//*' data='<a data-query-add=\'\\"sort=recent\\"\' h...'>,
 <Selector xpath='//*' data='<div class=\'\\"separator\\"\'></div>'>,
 <Selector xpath='//*' data='<a data-query-add=\'\\"sort=ranking\\"\' ...'>,
 <Selector xpath='//*' data='<div class=\'\\"separator\\"\'></div>'>,
 <Selector xpath='//*' data='<a data-query-add=\'\\"sort=salary\\"\' h...'>,
 <Selector xpath='//*' data='<h2 class=\'\\"ui\' left header style=\'\\...'>,
 <Selector xpath='//*' data='<span class=\'\\"desc\\"\'>검색 결과</span>'>,
 <Selector xpath='//*' data

In [1]:
data = pd.read_csv('test.csv')

In [51]:
req = requests.get("http://www.jobkorea.co.kr/Search/?stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&careerType=1&tabType=recruit&Page_No=1")
response = TextResponse(req.url, body=req.text, encoding='utf8')

In [64]:
response.xpath('//*[@id="dev_contents"]/div')

[]

# 잡코리아 크롤링
- 무조건 xpath 사용해서!!


조건 : 신입

In [259]:
import scrapy
from scrapy.http import TextResponse

In [260]:
# deadline 계산기 : 노 필요
import time
from datetime import date,timedelta
today = date.today()
deadline_date = today + timedelta(5)
"{}/{}마감".format(deadline_date.month,deadline_date.day)

'3/16마감'

In [ ]:
http://www.jobkorea.co.kr/Search/?stext=데이터%분석&careerType=1

In [66]:
# 전체 리스트 페이지 가져오기
req = requests.get("http://www.jobkorea.co.kr/Search/?stext=%EB%8D%B0%EC%9D%B4%ED%84%B0&careerType=1&tabType=recruit&Page_No=1")
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [67]:
links = response.xpath('//*[@id="content"]/div/div/div[1]/div/div[2]/div[2]/div/div[1]/ul/li/div/div[2]/a/@href').extract()
links = ["http://www.jobkorea.co.kr/" + link for link in links]
links

['http://www.jobkorea.co.kr//Recruit/GI_Read/31276276?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31239233?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31239183?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31294566?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31248438?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/30838023?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31271918?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31223077?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31217414?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31106375?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31278536?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31129588?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31060718?Oem_Code=C

In [68]:
# 각 공고별 링크 불러오기
links = [ "http://www.jobkorea.co.kr/" + link for link in response.xpath('//*[@id="content"]/div/div/div[1]/div/div[2]/div[2]/div/div[1]/ul/li/div/div[2]/a/@href').extract()]
links

['http://www.jobkorea.co.kr//Recruit/GI_Read/31276276?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31239233?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31239183?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31294566?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31248438?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/30838023?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31271918?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31223077?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31217414?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31106375?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31278536?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31129588?Oem_Code=C1&logpath=1',
 'http://www.jobkorea.co.kr//Recruit/GI_Read/31060718?Oem_Code=C

In [168]:
# 링크에서 공고하나 선택
url = links[3]
req = requests.get(url)
response = TextResponse(req.url,body=req.text,encoding='utf-8')

In [169]:
# 1. company name
company_name = response.xpath('//*[@id="container"]/section/div/article/div[1]/h3/span/text()')[0].extract().strip()
company_name

'한국산업기술시험원'

In [110]:
# 2. deadline
try:
    dead_line = response.xpath('//*[@id="tab02"]/div/article[1]/div/dl[2]/dd[2]/span/text()')[0].extract()[5:] + " 마감"
    print(dead_line)
except:
    print("상시 채용")

03.31 마감


In [111]:
# 3. business
url = "http://www.jobkorea.co.kr" + response.xpath('//*/article[contains(@class, "artReadCoInfo") and contains(@class, "divReadBx")]/div/div/p/a/@href')[0].extract()
req = requests.get(url)
response_detail_page = TextResponse(req.url,body=req.text,encoding='utf-8')
response_detail_page.xpath('//*[@id="company-body"]/div[1]/div[1]/div/div/div[9]/div[2]/div/div/text()')[0].extract()

'서비스/온라인정보제공업'

In [112]:
# 4. link
links[0]

'http://www.jobkorea.co.kr//Recruit/GI_Read/31276276?Oem_Code=C1&logpath=1'

In [113]:
# 5. position
response.xpath('//*[@id="container"]/section/div/article/div[1]/h3/text()')[1].extract().strip()

'Global Music 데이터 분석가 채용연계형 인턴십 모집'

In [114]:
# 6. job_condition
try:
    job_condition = response_detail_page.xpath('//*[@id="company-body"]/div[1]/div[1]/div/div/div[8]/div[2]/div/div/div/div/text()')[0].extract()
    print(job_condition)
except:
    print(0)

3,500만원


In [115]:
# 7. keyword
keyword = response.xpath('//*[@id="artKeywordSearch"]/ul/li/button/text()').extract()[:-1]
keyword

['기획·전략·경영',
 '기획',
 '응용프로그래머',
 'Python',
 'SQL',
 '빅데이터·AI(인공지능)',
 '데이터분석',
 '지표',
 '통계',
 '포털·컨텐츠·커뮤니티']

In [170]:
# 8. location
",".join(response.xpath('//*[@id="container"]/section/div/article/div[2]/div/dl/dd/a/text()').extract())

'경남 진주시,경기도 안산시 상록구'

In [258]:
# BeautifulSoup dom

In [21]:
# page_no를 돌려라
req = requests.get("http://www.jobkorea.co.kr/Search/?stext=%EB%8D%B0%EC%9D%B4%ED%84%B0&careerType=1&tabType=recruit&Page_No=1")
dom = BeautifulSoup(req.text, "html.parser")

In [282]:
# company 
dom.select('.name.dev_view')[0].get("title")

'㈜NAVER'

In [250]:
# description
response = requests.get(link)
dom_for_pages = BeautifulSoup(response.text, "html.parser")

In [251]:
dom_for_pages.select_one('.hd_4 ~ dl.tbList').text.strip('\n ')

'경력\n\n신입·경력\n\n학력\n\n학력무관 \n우대\n\n\n기본우대\n\n유관업무 경험자(인턴·알바)'

In [252]:
# service
dom_for_pages.select_one("#container > section > div > article > div.tbRow.clear > div.tbCol.tbCoInfo > dl.tbList").text.split('\n')

['', '기업명', '11번가 주식회사', '기업형태', '대기업 계열사·자회사', '상장여부', '비상장', '']

In [36]:
# position
dom.select('.post-list-info > a')[0].get('title')

'Global Music 데이터 분석가 채용연계형 인턴십 모집'

In [249]:
# link
link = "http://www.jobkorea.co.kr" + dom.select('.post-list-info > a')[15].get('href')

In [284]:
# deadline
dom.select('.post-list-info > p[class="option"]')[0].select_one('span[class=date]').text.strip('(~)월화수목금토일')+"마감"

'03/31마감'

In [47]:
# task & skills
dom.select('p.etc')[0].text

'데이터분석, 통계, 지표, 기획, Python, SQL'

In [95]:
# jon_condition
req2 = requests.get("http://www.jobkorea.co.kr/Salary/SearchOfSalary/671394")
dom = BeautifulSoup(req2.text, "html.parser")
salary = dom.select_one('div.salary-table-item.salary-table-newcomer > div.salary > div.value').text
salary

'3,500'

# 사람인

In [734]:
req = requests.get("http://m.saramin.co.kr/search?searchType=search&searchword=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EC%82%AC%EC%9D%B4%EC%96%B8%EC%8A%A4&q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EC%82%AC%EC%9D%B4%EC%96%B8%EC%8A%A4")
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [735]:
links = response.xpath('//*[@id="recruit_result"]/ul/li/div/a/@href').extract()
links = ["http://m.saramin.co.kr" + link for link in links]

In [821]:
req = requests.get(links[10])
response = TextResponse(req.url,body=req.text,encoding='utf8')
response

<200 http://m.saramin.co.kr/job-search/view?rec_idx=37603442&cn=search&cns=index&t_category=search&t_content=generic&searchword=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EC%82%AC%EC%9D%B4%EC%96%B8%EC%8A%A4&searchType=search&paid_fl=n>

In [822]:
# company_name 
company_name = response.xpath('//*[@id="card_set"]/div[1]/div/div[1]/div/span/a/text()')[0].extract()
company_name

'(주)라이언앤코'

In [823]:
# business : 미해결
response.xpath('//*[@id="card_set"]/div[1]/div/section[2]/div[1]/div[1]/div/dl[1]/dd/span').extract()

[]

In [824]:
# position
response.xpath('//*[@id="card_set"]/div[1]/div/div[1]/div/h2/text()')[0].extract().strip()

'[경력/신입] 경영컨설턴트 모집 [주 4일 근무제 도입 회사]'

In [825]:
# link
link = links[10]
print(link)

http://m.saramin.co.kr/job-search/view?rec_idx=37603442&cn=search&cns=index&t_category=search&t_content=generic&searchword=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EC%82%AC%EC%9D%B4%EC%96%B8%EC%8A%A4&searchType=search&paid_fl=n


In [849]:
# salary_condition
# 아래에 추가 항목이 있어서 전체 리스트로 받아와서 str변환
salary_condition = str(response.xpath('//*[@class="card_cont wrap_recruit_view"]/section[@class="section_view section_basic_view"]/div[@class="wrap_info_job"]/dl[2]/dd/*/text()').extract()).strip("[]''")
salary_condition

"면접 후 결정', '성과급 30% ~ 100% 별도"

In [862]:
# location
location = response.xpath('//*[@class="card_cont wrap_recruit_view"]/section[@class="section_view section_basic_view"]/div[@class="wrap_info_job"]/dl[3]/dd/div/text()')[0].extract().strip()
location

'서울 강남구'

In [827]:
# deadline
deadline = response.xpath('//*[@class="card_cont wrap_recruit_view"]/section[@class="section_view section_basic_view"]/div[@class="wrap_info_job"]/dl[1]/dd/span[1]/text()')[0].extract().strip()
deadline

'~03/16(월)'

In [828]:
# keyword
keyword = str(response.xpath('//*[@class="list_relation_tag"]/li/a/text()').extract()).strip("[]''")
keyword

"전문직', '경영분석·컨설턴트', '컨설턴트', '경영컨설팅', 'HR컨설팅', '서울', '강남구"